In [2]:
import pandas as pd
import numpy as np

In [3]:
movie_df = pd.read_csv("movies.csv")
rating_df = pd.read_csv("ratings.csv")


In [4]:
import re

def clean_title(title):
    return re.sub("[^a-zA-Z0-9 ]", "",title)

movie_df["clean_title"] = movie_df["title"].apply(clean_title)
movie_df


,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995
...,...,...,...,...
62418,209157,We (2018),Drama,We 2018
62419,209159,Window of the Soul (2001),Documentary,Window of the Soul 2001
62420,209163,Bad Poems (2018),Comedy|Drama,Bad Poems 2018
62421,209169,A Girl Thing (2001),(no genres listed),A Girl Thing 2001


In [5]:
movie_id = 3114

similar_user = rating_df[(rating_df["movieId"] == movie_id) & (rating_df["rating"] > 4)] ["userId"].unique()

In [6]:
similar_rating = rating_df[(rating_df["userId"].isin(similar_user)) & (rating_df["rating"] > 4)]["movieId"]

In [7]:
similar = similar_rating.value_counts() / len(similar_user)
similar = similar[similar > .1]
similar

3114     1.000000
1        0.651788
318      0.451418
260      0.440567
1196     0.431443
           ...   
1079     0.101850
91529    0.101480
903      0.101480
48394    0.100370
590      0.100370
Name: movieId, Length: 174, dtype: float64

In [8]:
all_user =  rating_df[(rating_df["movieId"].isin(similar.index)) & (rating_df["rating"] > 4 )]
all_user_rec = all_user["movieId"].value_counts() / len(all_user["userId"].unique())
all_user_rec

318     0.338189
296     0.281320
2571    0.241159
356     0.232494
593     0.223247
          ...   
2081    0.021354
3751    0.021170
2761    0.020680
1907    0.019796
3175    0.019417
Name: movieId, Length: 174, dtype: float64

In [9]:
compare_percentage = pd.concat([similar,all_user_rec],axis=1)
compare_percentage.columns = ["similar","all"]


In [10]:
compare_percentage["score"] = compare_percentage["similar"]/ compare_percentage["all"]
compare_percentage = compare_percentage.sort_values("score", ascending=False)


In [11]:
compare_percentage.head(10).merge(movie_df, left_index=True,right_on = "movieId")


,similar,all,score,movieId,title,genres,clean_title
3021,1.000000,0.053073,18.841924,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 2 1999
2264,0.203576,0.024796,8.210086,2355,"Bug's Life, A (1998)",Adventure|Animation|Children|Comedy,Bugs Life A 1998
2669,0.142047,0.020680,6.868954,2761,"Iron Giant, The (1999)",Adventure|Animation|Children|Drama|Sci-Fi,Iron Giant The 1999
14813,0.225771,0.034717,6.503216,78499,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX,Toy Story 3 2010
3650,0.132799,0.021170,6.272875,3751,Chicken Run (2000),Animation|Children|Comedy,Chicken Run 2000
1992,0.118126,0.021354,5.531892,2081,"Little Mermaid, The (1989)",Animation|Children|Comedy|Musical|Romance,Little Mermaid The 1989
1818,0.106165,0.019796,5.362941,1907,Mulan (1998),Adventure|Animation|Children|Comedy|Drama|Musi...,Mulan 1998
2895,0.135758,0.025378,5.349396,2987,Who Framed Roger Rabbit? (1988),Adventure|Animation|Children|Comedy|Crime|Fant...,Who Framed Roger Rabbit 1988
0,0.651788,0.123259,5.287943,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
3082,0.102589,0.019417,5.283613,3175,Galaxy Quest (1999),Adventure|Comedy|Sci-Fi,Galaxy Quest 1999


In [12]:
def similar_movies(movie_id):
    similar_user = rating_df[(rating_df["movieId"] == movie_id) & (rating_df["rating"] > 4)] ["userId"].unique()
    similar_rating = rating_df[(rating_df["userId"].isin(similar_user)) & (rating_df["rating"] > 4)]["movieId"]
 
    similar = similar_rating.value_counts() / len(similar_user)
    similar = similar[similar > .1]
    
    all_user =  rating_df[(rating_df["movieId"].isin(similar.index)) & (rating_df["rating"] > 4 )]
    all_user_rec = all_user["movieId"].value_counts() / len(all_user["userId"].unique())

    compare_percentage = pd.concat([similar,all_user_rec],axis=1)
    compare_percentage.columns = ["similar","all"]

    compare_percentage["score"] = compare_percentage["similar"]/ compare_percentage["all"]
    compare_percentage = compare_percentage.sort_values("score", ascending=False)

    compare_percentage.head(10).merge(movie_df, left_index=True,right_on = "movieId")[["score","title"]]


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(movie_df["clean_title"])

In [14]:
import ipywidgets as widgests
from IPython.display import display
from sklearn.metrics.pairwise import cosine_similarity

def search(title):
    title = clean_title(title)
    q = vectorizer.transform([title])
    similarity = cosine_similarity(q,tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5 :]
    results = movie_df.iloc[indices][::-1]
    return results

In [15]:
def similar_movies(movie_id):
    similar_user = rating_df[(rating_df["movieId"] == movie_id) & (rating_df["rating"] > 4)] ["userId"].unique()
    similar_rating = rating_df[(rating_df["userId"].isin(similar_user)) & (rating_df["rating"] > 4)]["movieId"]
 
    similar = similar_rating.value_counts() / len(similar_user)
    similar = similar[similar > .1]
    
    all_user =  rating_df[(rating_df["movieId"].isin(similar.index)) & (rating_df["rating"] > 4 )]
    all_user_rec = all_user["movieId"].value_counts() / len(all_user["userId"].unique())

    compare_percentage = pd.concat([similar,all_user_rec],axis=1)
    compare_percentage.columns = ["similar","all"]

    compare_percentage["score"] = compare_percentage["similar"]/ compare_percentage["all"]
    compare_percentage = compare_percentage.sort_values("score", ascending=False)

    compare_percentage.head(10).merge(movie_df, left_index=True,right_on = "movieId")[["score","title"]]




movie_input = widgests.Text(
    value = "",
    description = "Movie Title:",
    disabled = False
)

recommendation_list = widgests.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = search(title)
            movie_id = results.iloc[0]["movieId"]
            display(similar_movies(movie_id))

movie_input.observe(on_type, names="value")
display(movie_input,recommendation_list)

Text(value='', description='Movie Title:')

Output()

In [16]:
def find_similar_movies(movie_id):
    similar_users = rating_df[(rating_df["movieId"] == movie_id) & (rating_df["rating"] > 4)]["userId"].unique()
    similar_user_recs = rating_df[(rating_df["userId"].isin(similar_users)) & (rating_df["rating"] > 4)]["movieId"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    all_users = rating_df[(rating_df["movieId"].isin(similar_user_recs.index)) & (rating_df["rating"] > 4)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]
    
    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(movie_df, left_index=True, right_on="movieId")[["score", "title", "genres"]]

In [17]:
import ipywidgets as widgets
from IPython.display import display

movie_name_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = search(title)
            movie_id = results.iloc[0]["movieId"]
            display(find_similar_movies(movie_id))

movie_name_input.observe(on_type, names='value')

display(movie_name_input, recommendation_list)

Text(value='Toy Story', description='Movie Title:')

Output()